In [7]:

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
from scipy.linalg import expm
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity
from qiskit import QuantumCircuit

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_heisenberg_qutip

np.random.seed(667)

In [8]:

#! t_0 = 1, w_0 = 2pi/N

num_qubits = 3
num_energy_bits = 6
eps = 0.1
sigma = 5
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1
T = 1
total_time = T
num_trotter_steps = 100

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)

step_size = total_time / num_trotter_steps
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, 
                                      coeffs=hamiltonian.rescaled_coeffs, shift=(total_time*hamiltonian.shift))

Original spectrum:  [-3.7    -3.177  -2.5482 -1.7     1.625   2.4547  3.2521  3.7936]
Ideal spectrum:  [0.     0.0314 0.0692 0.1201 0.3198 0.3696 0.4175 0.45  ]
Nonrescaled coefficients:  [1.  0.8 0.9 1. ]
Rescaled coefficients:  [0.06005156 0.04804125 0.05404641 0.06005156]
eXX, eYY, eZZ for qubit (0, 1)
eXX, eYY, eZZ for qubit (1, 2)
eXX, eYY, eZZ for qubit (2, 0)


In [9]:
N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))

with open(f'/Users/bence/code/liouvillian_metro/data/block_initial_state_n{num_qubits}k{num_energy_bits}.pkl', 'rb') as f:
    initial_state = pickle.load(f).data.reshape(2**(num_qubits), 1)


In [10]:
def energy_state_for_bits(bitstring: str) -> np.ndarray:
    """Convert a bitstring to an energy eigenstate."""
    tensor_list = []
    for bit in bitstring:
        if bit == '0':
            tensor_list.append(qt.basis(2, 0))
        else:
            tensor_list.append(qt.basis(2, 1))
    state = qt.tensor(tensor_list).full()
    return state / np.linalg.norm(state)

def projector_to_energy_eigenspace(bitstring: str) -> np.ndarray:
    """Return the projector to the energy eigenstate corresponding to a bitstring."""
    state = energy_state_for_bits(bitstring)
    return state @ state.conj().T

# def energy_basis_matrix(num_energy_bits: int) -> np.ndarray:
    
#     all_energy_bitstrings = [f'{i:0{num_energy_bits}b}' for i in range(2**num_energy_bits)]
#     all_energy_states = [energy_state_for_bits(bitstring).reshape(1, 2**num_energy_bits) for bitstring in all_energy_bitstrings]
#     return np.concatenate(all_energy_states, axis=0)


In [11]:
# Test projection into eigenspace
# coeffs = np.sqrt(np.array([0.25, 0.75]))
# print(f'Coeffs {coeffs}')
# energy_state_in_circuit = coeffs[0] * energy_state_for_bits('000') + coeffs[1] * energy_state_for_bits('001')
# projector = projector_to_energy_eigenspace('011')
# projected_state = projector @ energy_state_in_circuit
# print(np.linalg.norm(projected_state))


In [12]:
with open(f'/Users/bence/code/liouvillian_metro/data/block_state_before_measure_n{num_qubits}k{num_energy_bits}.pkl', 'rb') as f:
    circ_state = pickle.load(f).data.reshape(2**(num_qubits + num_energy_bits + 1), 1)
    
phase_bits = '010011'
if len(phase_bits) != num_energy_bits:
    raise ValueError('This is not the right amount of phase bits')
if phase_bits[0] == '1':
    phase = (int(phase_bits[1:], 2) - 2**(num_energy_bits - 1)) # exp(i 2pi phase)
else:
    phase = int(phase_bits[1:], 2)

energy = 2 * np.pi * phase / 2**num_energy_bits
print(f'Energy: {energy}')

projector_to_boltz_zero = qt.tensor([qt.qeye(2**(num_qubits + num_energy_bits)), qt.basis(2, 0).dag()]).full()
projected_circ_state_0 = projector_to_boltz_zero @ circ_state
projected_circ_state_0 /= np.linalg.norm(projected_circ_state_0)

projector_to_energy = lambda bitstring: qt.tensor([qt.qeye(2**num_qubits), qt.Qobj(energy_state_for_bits(bitstring))]).dag().full()
projected_circ_state_0w = projector_to_energy(phase_bits) @ projected_circ_state_0
projected_circ_state_0w /= np.linalg.norm(projected_circ_state_0w)

#* --- 
jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()
oft_op = oft(jump_op, phase, 2**num_energy_bits, sigma, trotter=trott_U_qt.full())

boltzmann = lambda beta, w: np.min([1, np.exp(-beta * w)])
state_we_should_get = lambda w: np.sqrt(boltzmann(beta, w))*(oft_op @ initial_state)

state_we_def_should_get = state_we_should_get(energy).reshape(2**num_qubits, 1)
state_we_def_should_get /= np.linalg.norm(state_we_def_should_get)
# print(state_we_def_should_get)
# print(projected_circ_state_0w)

dist = qt.tracedist(qt.Qobj(projected_circ_state_0w), qt.Qobj(state_we_def_should_get))
print(f'Trace distance: {dist}')
fid = state_fidelity(projected_circ_state_0w, state_we_def_should_get)
print(f'Fidelity {fid}')

# projected_circ_sys_dm = partial_trace(projected_circ_state, list(range(num_energy_bits))).data
# projected_circ_sys_dm /= np.trace(projected_circ_sys_dm)


Energy: 1.8653206380689396
Trace distance: 0.9940373669185191
Fidelity 0.011893161524510159
